In [2]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, Queue
import random
import math

In [45]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=10000.0):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(inputs=self.inputT, filters=30, kernel_size=[1,3], activation=tf.nn.leaky_relu)
      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        #self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=150, kernel_size=[1,8], activation=tf.nn.relu)
        #self.conv3 = tf.layers.conv2d(inputs = self.conv2, filters=200, kernel_size=[1,41], activation=tf.nn.relu)
        #self.conv4 = tf.layers.conv2d(inputs=self.conv3, filters=30, kernel_size=[1,1] , activation=tf.nn.relu)
        
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=30, kernel_size=[1,48], activation=tf.nn.leaky_relu)
        
        
        self.hidden = tf.layers.dense(self.conv2, 2000, activation=tf.nn.leaky_relu)
        #self.drop = tf.layers.dropout(self.hidden)
        self.final2 = tf.layers.dense(self.hidden, 1)
        self._allocate = tf.nn.softmax(self.final2, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        #self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))

        self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(self.priceChanges), tf.reshape(self._allocate, [tf.shape(self._allocate)[0], numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run(self.averageLoss, feed_dict={self.inputT : obs, self.priceChanges: prices}))
        print(prices)

In [47]:
def main():
    feed = feedBackup.copy()
    seeds = [3, 5, 7]
    tf.reset_default_graph()
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    tf.set_random_seed(seeds[1])
    test1 = Agent(len(testSim.currencies), 50, sess)
    sess.run(tf.global_variables_initializer())

    testData = feed[0]
    feed.pop(0)

    #print(testData)


    try:
        for episode in range(500):
            print("Episode: {}".format(episode))
            index = list(range(len(feed)))
            loss = 0
            count = len(feed)
            while len(index) != 0:
                for i in random.sample(index, 1):
                    with tf.device('/gpu:0'):
                        loss += test1.train_step(feed[i][0], feed[i][1])
                    index.remove(i)
            print(loss/count)
    except KeyboardInterrupt:
        print("Ending")
        test1.test_model(testData[0], testData[1])
    finally:
        print("Ending")
        test1.test_model(testData[0], testData[1])
            
if __name__ == "__main__":
    main()

Episode: 0
9999.98037997159
Episode: 1
9999.98037997159
Episode: 2
9999.980291193182
Episode: 3
9999.979669744318
Episode: 4
9999.980202414772
Episode: 5
9999.979758522728
Episode: 6
9999.980291193182
Episode: 7
9999.980113636364
Episode: 8
9999.98037997159
Episode: 9
9999.980202414772
Episode: 10
9999.979936079546
Episode: 11
9999.980291193182
Episode: 12
9999.980024857954
Episode: 13
9999.980291193182
Episode: 14
9999.980202414772
Episode: 15
9999.980024857954
Episode: 16
9999.980024857954
Episode: 17
9999.980024857954
Episode: 18
9999.979847301136
Episode: 19
9999.980202414772
Episode: 20
9999.980113636364
Episode: 21
9999.979847301136
Episode: 22
9999.980202414772
Episode: 23
9999.98037997159
Episode: 24
9999.980113636364
Episode: 25
9999.980202414772
Episode: 26
9999.980113636364
Episode: 27
9999.980113636364
Episode: 28
9999.98046875
Episode: 29
9999.980024857954
Episode: 30
9999.980291193182
Episode: 31
9999.980646306818
Episode: 32
9999.980735085228
Episode: 33
9999.98073508522

10000.097123579546
Episode: 269
10000.097123579546
Episode: 270
10000.097212357954
Episode: 271
10000.09765625
Episode: 272
10000.09765625
Episode: 273
10000.097389914772
Episode: 274
10000.09765625
Episode: 275
10000.09765625
Episode: 276
10000.09765625
Episode: 277
10000.09765625
Episode: 278
10000.097922585228
Episode: 279
10000.097922585228
Episode: 280
10000.097833806818
Episode: 281
10000.098011363636
Episode: 282
10000.097833806818
Episode: 283
10000.09774502841
Episode: 284
10000.09774502841
Episode: 285
10000.097833806818
Episode: 286
10000.097833806818
Episode: 287
10000.098277698864
Episode: 288
10000.098188920454
Episode: 289
10000.098188920454
Episode: 290
10000.098011363636
Episode: 291
10000.098100142046
Episode: 292
10000.098188920454
Episode: 293
10000.098100142046
Episode: 294
10000.098188920454
Episode: 295
10000.098188920454
Episode: 296
10000.098455255682
Episode: 297
10000.098366477272
Episode: 298
10000.098188920454
Episode: 299
10000.098188920454
Episode: 300
10

In [48]:
def importData(simulator):
    testSim = simulator
    q = Queue()
    jobs = []
    PERIOD_SIZE = 50
    BATCH_SIZE = 10
    minSize = math.inf
    for pair in testSim.df.values():
        if len(pair.index) < minSize:
            minSize = len(pair.index)
    print(minSize)
    #BATCH_COUNT = int(math.floor(minSize / BATCH_SIZE))
    BATCH_COUNT = 12
    print(BATCH_COUNT)
    BATCH_OFFSET = 10
    dates = testSim.getAllDates()
    index = list(range(BATCH_COUNT))
    feed = []
    threads = 12

    running = False
    count = 0
    while 1:
        if count < threads:
            for i in random.sample(index, threads-count if len(index) >= threads-count else len(index)):
                p = Process(target=testSim.processTimePeriod, args=(q, PERIOD_SIZE, dates, BATCH_SIZE * (i + BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE))
                jobs.append(p)
                print("Starting")
                p.start() 
                index.remove(i)
        count = 0
        for p in jobs:
            if not p.is_alive():
                p.terminate()
                jobs.remove(p)
            else:
                count += 1
        while not q.empty():
            print('Getting')
            feed.append(q.get())
        if count == 0 and len(index) == 0 and len(feed) == BATCH_COUNT:
            print('empty')
            break
    print(len(feed))
    return feed

In [17]:
feedBackup = importData(testSim)

586243
12
Starting
Starting
Starting
0
0
Starting
0
Starting
Starting
0
0
Starting
0
Starting
Starting
0
Starting
0
0
Starting
Starting
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Getting
Getting
Getting
Getting
Getting
Getting
Getting
Getting
Getting
Getting
Getting
Getting
empty
12


In [5]:
testSim = Market(['EUR','USD', 'TRY', 'JPY'], os.path.abspath('../Data_Processing/ProcessedData'))